In [21]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import sys
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import random
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import shutil
import pdb

sys.path.append('../')
from meta_fusion.benchmarks_general import *
from meta_fusion.methods_general import *
from meta_fusion.models_general import *
from meta_fusion.utils import *
from meta_fusion.third_party import *
from meta_fusion.synthetic_data_general import PrepareSyntheticData
from meta_fusion.synthetic_data_general import PrepareSyntheticData

from meta_fusion.config import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [89]:
def run_single_experiment(config, extractor_config, 
                          n, dim_modalities, dim_latent, noise_ratios, trans_type, mod_prop, interactive_modalities, interactive_prop,
                          mod_outs, combined_hiddens, mod_hiddens, is_mod_static, freeze_mod_extractors,
                          extractor_type, seperate, 
                          random_state, 
                          run_oracle=False, run_coop=True):

    config['random_state'] = random_state
    extractor_config['random_state'] = random_state
    res_list = []
    best_rho = {}
    cohort_pairs = {}

    #----------------#
    # Split dataset  #
    #----------------#
    train_loader, val_loader, test_loader, oracle_train_loader, oracle_val_loader, oracle_test_loader =\
    data_preparer.get_data_loaders(n, trans_type=trans_type, mod_prop=mod_prop, 
                                   interactive_prop = interactive_prop, interactive_modalities = interactive_modalities,
                                   dim_modalities=dim_modalities, dim_latent=dim_latent,
                                   noise_ratios=noise_ratios, random_state=random_state)
    # Get data info
    data_info = data_preparer.get_data_info()
    dim_modalities = data_info[0]
    n = data_info[1]
    n_train = data_info[2]
    n_val = data_info[3]
    n_test = data_info[4]

    print(f"Finished splitting {data_name} dataset. Data information are summarized below:\n"
            f"Modality dimensions: {dim_modalities}\n"
            f"Data size: {n}\n"
            f"Train size: {n_train}\n"
            f"Val size: {n_val}\n"
            f"Test size: {n_test}")
    sys.stdout.flush() 

    #------------------#
    # Benchmark models #
    #------------------#
    bm_outs = [[d,0] for d in dim_modalities]
    bm_extractor = Extractors(bm_outs, dim_modalities, train_loader, val_loader)
    _ = bm_extractor.get_dummy_extractors()
    bm_cohort = Cohorts(extractors=bm_extractor, combined_hidden_layers=combined_hiddens, output_dim=output_dim)

    if run_oracle:
        dim_oracles = dim_latent[:-1].copy()
        dim_oracles[-1]+=dim_latent[-1]
        oracle_outs = [[d,0] for d in dim_oracles]
        oracle_extractor = Extractors(oracle_outs, dim_oracles, oracle_train_loader, oracle_val_loader)
        _ = oracle_extractor.get_dummy_extractors()
        oracle_cohort = Cohorts(extractors=oracle_extractor, combined_hidden_layers=combined_hiddens, output_dim=output_dim)

    #----------------------------#
    # Proposed model: MetaFuse   #
    #----------------------------#
    meta_extractor = Extractors(mod_outs, dim_modalities, train_loader, val_loader)
    if extractor_type == 'encoder':
        _ = meta_extractor.get_encoder_extractors(mod_hiddens, seperate=seperate, config=extractor_config)
    elif extractor_type == 'PCA':
        _ = meta_extractor.get_PCA_extractors()
    meta_cohort = Cohorts(extractors=meta_extractor, combined_hidden_layers=combined_hiddens, output_dim=output_dim,
                          is_mod_static=is_mod_static, freeze_mod_extractors=freeze_mod_extractors)


    #------------------------------#
    #  Train and test benchmarks   #
    #------------------------------#
    bm_models = bm_cohort.get_cohort_models()
    _, bm_dims = bm_cohort.get_cohort_info()
    bm = Benchmarks(config, bm_models, bm_dims, [train_loader, val_loader])
    bm.train()
    res = bm.test(test_loader)
    res_list.append(res)
    print(f"Finished running basic benchmarks!")
    
    if run_oracle:
        oracle_config = config
        oracle_config["init_lr"] = 0.001
        oracle_models = oracle_cohort.get_cohort_models()
        _, oracle_dims = oracle_cohort.get_cohort_info()
        oracle = Benchmarks(config, oracle_models, oracle_dims, [oracle_train_loader, oracle_val_loader])
        oracle.train()
        res = oracle.test(oracle_test_loader)
        res = {f"oracle_{key}": value for key, value in res.items()}
        res_list.append(res)
        print(f"Finished running oracle benchmarks!")
        
    if run_coop:
        if len(dim_modalities) == 2:
            bm_models = bm_cohort.get_cohort_models()
            _, bm_dims = bm_cohort.get_cohort_info()    
            coop = Coop(config, bm_models, bm_dims, [train_loader, val_loader])
            coop.train()
            res = coop.test(test_loader)
            res_list.append(res)
            best_rho['coop'] = coop.best_rho
            print(f"Finished running coop!")
        else:
            print(f"Skip coop benchmark since it is not clear how it handles multiple modalities with interactions.")

    
    #------------------------------#
    #  Train and test MetaFuse  #
    #------------------------------#
    cohort_models = meta_cohort.get_cohort_models()
    _, dim_pairs = meta_cohort.get_cohort_info()
    metafuse = Trainer(config, cohort_models, [train_loader, val_loader])
    metafuse.train() 
    res = metafuse.test(test_loader)
    res_list.append(res)
    
    metafuse.train_ablation() 
    res = metafuse.test_ablation(test_loader)
    res_list.append(res)
    
    best_rho['meta_learner'] = metafuse.best_rho
    cohort_pairs['cohort'] = dim_pairs
    print(f"Finished running meta fusion!")
    
    results = []
    for res in res_list:
        for method, val in res.items():
            results.append({'Method': method, 'Test_metric': val, 
                            'best_rho':best_rho.get(method), 'cohort_pairs':cohort_pairs.get(method)})
    

    results = pd.DataFrame(results)

    results['random_state']=random_state
    results["dim_modalities"] = [dim_modalities] * len(results)
    results['n'] = n
    results['n_train'] = n_train
    results['n_val'] = n_val
    results['n_test'] = n_test 
    results['scale'] = scale

    return results

In [90]:
# Fixed data parameters
repetition=1
seed=1
scale=0

# Data model parameters
n = 2000
dim_modalities = [300, 200, 100, 50]
dim_latent = [20, 20, 10, 10, 0]
noise_ratios = [0.5, 0.1, 0.2, 0.1]
trans_type = ["quadratic", "linear", "linear", "linear","linear"]
mod_prop = [1, 1, 1, 1, 0, 0]
interactive_modalities = [1, 1, 1, 1]
interactive_prop = 0.1

mod_outs = [[50,30],[80,50,30],[30],[20]]
combined_hiddens = [300,200,100]
mod_hiddens = [[128]]*len(dim_modalities)

# n = 2000
# dim_modalities = [2000, 100]
# dim_latent = [20, 20, 0]
# noise_ratios = [0.5, 0.1]
# trans_type = ["quadratic", "linear", "linear"]
# mod_prop = [1, 1, 0, 1]
# interactive_prop = 0.1


# mod_outs = [[60],[40, 60]]
# combined_hiddens = [300,200,100]
# mod_hiddens = [[128]]*len(dim_modalities)

# data parameters
data_name = 'regression'
exp_name = data_name + "_" + "quadratic_early"
output_dim = 1  # specify the output dimension for regression


extractor_type = 'PCA'
seperate=False
is_mod_static=[True]*len(dim_modalities)
freeze_mod_extractors=[False]*len(dim_modalities)

# Load default model configurations 
config = load_config('../experiments_synthetic/config.json')
extractor_config = load_config('../experiments_synthetic/config_extractor.json')

# Model files directory
ckpt_dir = f"./checkpoints/{exp_name}/scale{scale}_seed{seed}/"
config['ckpt_dir'] = extractor_config['ckpt_dir'] = ckpt_dir

# Update other training parameters
config['divergence_weight_type'] = 'clustering'
config['optimal_k'] = None
config['output_dim'] = extractor_config['output_dim'] = output_dim
config["init_lr"] = 0.001
config["epochs"] = 10
extractor_config["init_lr"] = 0.001
config["ensemble_methods"] = [
        "simple_average",
        "weighted_average",
        "greedy_ensemble"
        ]
#####################
#    Load Dataset   #
#####################
data_preparer = PrepareSyntheticData(data_name = data_name, test_size = 0.2, val_size = 0.2)

In [91]:
results = []

#for i in tqdm(range(1, repetition+1), desc="Repetitions", leave=True, position=0):
for i in range(1, repetition+1):
    print(f'Running with repetition {i}...')
    random_state = repetition * (seed-1) + i
    set_random_seed(random_state)
    
    # Run experiment
    tmp = run_single_experiment(config, extractor_config, 
                              n, dim_modalities, dim_latent, noise_ratios, trans_type, mod_prop, interactive_modalities, interactive_prop,
                              mod_outs, combined_hiddens, mod_hiddens, is_mod_static, freeze_mod_extractors,
                              extractor_type, seperate, run_oracle=True, run_coop=False, random_state=random_state)
        
    results.append(tmp)

results = pd.concat(results, ignore_index=True)


#####################
#    Save Results   #
#####################
# results.to_csv(outfile, index=False)
# print("\nResults written to {:s}\n".format(outfile))
# sys.stdout.flush()

# After the job is done, remove the model directory to free up space
if os.path.exists(ckpt_dir):
    print(f"Deleting the model checkpoint directory: {ckpt_dir}")
    shutil.rmtree(ckpt_dir)
    print(f"Model checkpoint directory {ckpt_dir} has been deleted.")

Running with repetition 1...
Finished splitting regression dataset. Data information are summarized below:
Modality dimensions: [300, 200, 100, 50]
Data size: 2000
Train size: 1280
Val size: 320
Test size: 400
Start training benchmark models...
Training with disagreement penalty = 0

Epoch: 1/10 - LR: 0.001000


100%|█████████████████████████████████████████████████████████████████████████████| 1280/1280 [00:03<00:00, 349.76it/s]


model_1: train loss: 497.006, train task loss: 497.006 - val loss: 261.421, val task loss: 261.421 [*] Best so far
Created directory: ./checkpoints/regression_quadratic_early/scale0_seed1/0
model_2: train loss: 473.858, train task loss: 473.858 - val loss: 243.686, val task loss: 243.686 [*] Best so far
model_3: train loss: 489.301, train task loss: 489.301 - val loss: 289.694, val task loss: 289.694 [*] Best so far
model_4: train loss: 490.328, train task loss: 490.328 - val loss: 295.394, val task loss: 295.394 [*] Best so far
model_5: train loss: 444.354, train task loss: 444.354 - val loss: 158.570, val task loss: 158.570 [*] Best so far

Epoch: 2/10 - LR: 0.001000


100%|█████████████████████████████████████████████████████████████████████████████| 1280/1280 [00:03<00:00, 328.21it/s]


model_1: train loss: 256.575, train task loss: 256.575 - val loss: 230.286, val task loss: 230.286 [*] Best so far
model_2: train loss: 265.788, train task loss: 265.788 - val loss: 240.228, val task loss: 240.228 [*] Best so far
model_3: train loss: 319.710, train task loss: 319.710 - val loss: 284.617, val task loss: 284.617 [*] Best so far
model_4: train loss: 318.679, train task loss: 318.679 - val loss: 274.073, val task loss: 274.073 [*] Best so far
model_5: train loss: 151.980, train task loss: 151.980 - val loss: 144.855, val task loss: 144.855 [*] Best so far

Epoch: 3/10 - LR: 0.001000


...

model_1: train loss: 132.874, train task loss: 105.726 - val loss: 153.310, val task loss: 125.709 [*] Best so far
model_2: train loss: 84.029, train task loss: 79.436 - val loss: 132.099, val task loss: 118.232 [*] Best so far
model_3: train loss: 82.154, train task loss: 79.211 - val loss: 118.165, val task loss: 110.650 [*] Best so far
model_4: train loss: 89.671, train task loss: 84.552 - val loss: 136.031, val task loss: 122.953 [*] Best so far
model_5: train loss: 86.518, train task loss: 81.730 - val loss: 128.313, val task loss: 114.865 [*] Best so far
model_6: train loss: 84.534, train task loss: 81.591 - val loss: 124.692, val task loss: 117.177 [*] Best so far

Epoch: 9/10 - LR: 0.001000


100%|█████████████████████████████████████████████████████████████████████████████| 1280/1280 [00:06<00:00, 194.25it/s]


model_1: train loss: 119.059, train task loss: 94.548 - val loss: 159.267, val task loss: 127.442
model_2: train loss: 72.083, train task loss: 66.545 - val loss: 136.190, val task loss: 116.090 [*] Best so far
model_3: train loss: 68.746, train task loss: 65.043 - val loss: 119.871, val task loss: 109.289 [*] Best so far
model_4: train loss: 81.183, train task loss: 73.319 - val loss: 143.508, val task loss: 122.054 [*] Best so far
model_5: train loss: 72.993, train task loss: 67.230 - val loss: 132.727, val task loss: 113.688 [*] Best so far
model_6: train loss: 70.778, train task loss: 67.075 - val loss: 126.686, val task loss: 116.103 [*] Best so far

Epoch: 10/10 - LR: 0.001000


100%|█████████████████████████████████████████████████████████████████████████████| 1280/1280 [00:06<00:00, 188.75it/s]


model_1: train loss: 160.656, train task loss: 95.260 - val loss: 179.879, val task loss: 120.896 [*] Best so far
model_2: train loss: 63.132, train task loss: 54.365 - val loss: 135.285, val task loss: 105.327 [*] Best so far
model_3: train loss: 58.038, train task loss: 53.263 - val loss: 113.428, val task loss: 97.978 [*] Best so far
model_4: train loss: 73.644, train task loss: 60.864 - val loss: 140.970, val task loss: 109.802 [*] Best so far
model_5: train loss: 63.264, train task loss: 53.917 - val loss: 130.176, val task loss: 101.707 [*] Best so far
model_6: train loss: 56.146, train task loss: 51.371 - val loss: 116.536, val task loss: 101.086 [*] Best so far
Finished training student cohort!
Selecting the optimal disgreement penalty via cross-validation...
Best rho: 0.99 with average task loss: 89.6919
Done!
Selecting greedy ensemble on the best cohort...
Pruned 2 worst models, keeping 4 models
Initial best models: [5, 4, 2] with losses: [tensor(87.7970, grad_fn=<MseLossBack

In [92]:
# Convert tensor values in 'Test_MSE' column to float
def convert_to_float(value):
    if isinstance(value, torch.Tensor):
        return value.item()  # Convert tensor to float
    elif isinstance(value, list):
        return [v.item() if isinstance(v, torch.Tensor) else v for v in value]  # Convert list of tensors
    return value

In [93]:
import matplotlib.pyplot as plt
results['Test_metric'] = results['Test_metric'].apply(convert_to_float)

In [95]:
# Filter out methods you don't want to include
methods_to_include = ['early_fusion', 'late_fusion', "oracle_early_fusion", "oracle_late_fusion", 
                      'simple_average','weighted_average','meta_learner', 'greedy_ensemble',
                     'best_single', 'indep_best_single']  # Change this list based on your needs
df_filtered = results[results['Method'].isin(methods_to_include)]

In [96]:
# Mean and standard deviation of Test_MSE for each method
grouped_stats = df_filtered.groupby('Method')['Test_metric'].agg(['mean', 'std'])

print(grouped_stats)

                           mean  std
Method                              
best_single           91.989861  NaN
early_fusion         110.878769  NaN
greedy_ensemble       85.793556  NaN
indep_best_single    105.578903  NaN
late_fusion          202.205124  NaN
oracle_early_fusion   93.921539  NaN
oracle_late_fusion   186.947754  NaN
simple_average         86.19706  NaN
weighted_average      86.105324  NaN


In [94]:
results

,Method,Test_metric,best_rho,cohort_pairs,random_state,dim_modalities,n,n_train,n_val,n_test,scale
0,modality_1,227.999466,None,None,1,"[300, 200, 100, 50]",2000,1280,320,400,0
1,modality_2,214.094528,None,None,1,"[300, 200, 100, 50]",2000,1280,320,400,0
2,modality_3,281.724792,None,None,1,"[300, 200, 100, 50]",2000,1280,320,400,0
3,modality_4,271.296265,None,None,1,"[300, 200, 100, 50]",2000,1280,320,400,0
4,early_fusion,110.878769,None,None,1,"[300, 200, 100, 50]",2000,1280,320,400,0
5,late_fusion,202.205124,None,None,1,"[300, 200, 100, 50]",2000,1280,320,400,0
6,oracle_modality_1,198.725403,None,None,1,"[300, 200, 100, 50]",2000,1280,320,400,0
7,oracle_modality_2,210.792252,None,None,1,"[300, 200, 100, 50]",2000,1280,320,400,0
8,oracle_modality_3,280.873779,None,None,1,"[300, 200, 100, 50]",2000,1280,320,400,0
9,oracle_modality_4,265.844177,None,None,1,"[300, 200, 100, 50]",2000,1280,320,400,0
